## Meteo to nc

In [1]:
from Poseidon.utils import *
from Poseidon.meteo import *
from netCDF4 import Dataset
import datetime

### setup

In [200]:
#define in a dictionary the properties e.g
dic={'lon0':-35., # lat/lon window
     'lon1':43.,
     'lat0':24.5,
     'lat1':76.5}

### extract meteo forcing

In [201]:
#set meteo forcing
m = ecmwf() # initialize

In [203]:
# specify the forecast times ft1,ft2
m.ft1 = 0
m.ft2 = 14 #that is 0,1 hours

In [204]:
# parse the ecmwf grib file.
m.parse(path='/home/critechproc/STORAGE/meteo/2010/10/01/20101001.00.tropical_cyclone.grib',**dic)

100%|██████████| 42/42 [00:15<00:00,  2.52it/s]


meteo done


## output 

Below the output specifically for Schism but one can adapt appropriately

In [219]:
#from DataFile class create a file
d = DataFile('/home/critechproc/STORAGE/SCHISM/test1/sflux/sflux_air_1.001.nc', write=True, create=True,format='NETCDF3_64BIT')

In [220]:
## add OPTIONAL attributes to the nc file
info = {'description' : '',
    'history' :'JRC Ispra European Commission',
    'source' : 'netCDF4 python module'}

In [221]:
d.info(**info)

In [222]:
tstamp = datetime.datetime(2013,10,1,0)

In [223]:
tu = datetime.datetime.strftime(tstamp,'%Y-%m-%d' )
#bd = datetime.datetime.strftime(tstamp,'[%Y  %-m  %-d  %-H]' )

In [224]:
bd = np.array([tstamp.year,tstamp.month,tstamp.day,tstamp.hour])

In [225]:
time = np.arange(m.ft1,m.ft2) /24.

In [226]:
tt=time[:].astype(np.float32)
pp=m.p[:,:,:].astype(np.float32)
uu=m.u[:,:,:].astype(np.float32)
vv=m.v[:,:,:].astype(np.float32)
lats=m.lats.astype(np.float32)
lons=m.lons.astype(np.float32)

In [227]:
spfh = np.empty(vv.shape)

In [228]:
#Write variables
d.write('time',tt, ['time'], **{'limited':False,'long_name': 'Time', 'standard_name': 'time', 'units': tu,'base_date': bd})


d.write('lat',lats, ['nx_grid','ny_grid'], **{'long_name': 'Latitude', 'standard_name': 'latitude', 'units': 'degrees_north'})
d.write('lon',lons, ['nx_grid','ny_grid'], **{'long_name': 'Longitude', 'standard_name': 'longitude', 'units': 'degrees_east'})
d.write('prmsl',pp, ['time','nx_grid','ny_grid'], **{'long_name': 'Pressure reduced to MSL', 'standard_name': 'air_pressure_at_sea_level', 'units': 'Pa'})
d.write('uwind',uu, ['time','nx_grid','ny_grid'], **{'long_name': 'Surface Eastward Air Velocity', 'standard_name': 'eastward_wind', 'units': 'm/s'})
d.write('vwind',vv, ['time','nx_grid','ny_grid'], **{'long_name': 'Surface Northward Air Velocity', 'standard_name': 'northward_wind', 'units': 'm/s'})
d.write('spfh',spfh, ['time','nx_grid','ny_grid'], **{'long_name': 'Surface Specific Humidity (2m AGL)', 'standard_name': 'specific_humidity', 'units': '1'})
d.write('stmp',spfh, ['time','nx_grid','ny_grid'], **{'long_name': 'Surface Temperature', 'standard_name': 'surface temperature', 'units': 'degrees'})

In [229]:
#close
d.close()

In [260]:
#check
q1 = Dataset('/home/critechproc/STORAGE/SCHISM/test1/sflux/sflux_air_1.001.nc')

In [264]:
q1.variables['time']

<type 'netCDF4._netCDF4.Variable'>
float32 time(time)
    units: 2010-02-01
    standard_name: time
    base_date: [2010    2    1    0]
    long_name: Time
unlimited dimensions: time
current shape = (13,)
filling off

In [282]:
#check
q2 = Dataset('/home/critechproc/STORAGE/SCHISM/test1/sflux/sflux_air_1.002.nc')

In [283]:
q2.variables['time']

<type 'netCDF4._netCDF4.Variable'>
float32 time(x)
    units: 2010-02-01
    standard_name: time
    base_date: [2010    2    1   12]
    long_name: Time
unlimited dimensions: x
current shape = (13,)
filling off

## results

In [274]:
#check
r = Dataset('/home/critechproc/STORAGE/SCHISM/test1/outputs/schism_history.nc')

In [275]:
r.variables.keys()

[u'ocean_time',
 u'ocean_time_day',
 u'ocean_time_str',
 u'kbp00',
 u'idry',
 u'WATLEV',
 u'davx',
 u'davy']

In [277]:
r.variables['WATLEV']

<type 'netCDF4._netCDF4.Variable'>
float32 WATLEV(ocean_time, np_global)
unlimited dimensions: ocean_time
current shape = (1, 175156)
filling off

In [279]:
r.variables['idry']

<type 'netCDF4._netCDF4.Variable'>
int32 idry(ocean_time, np_global)
unlimited dimensions: ocean_time
current shape = (1, 175156)
filling off

In [280]:
#check
r1 = Dataset('/home/critechproc/STORAGE/SCHISM/test1/sflux/sflux_air_1.002.nc')

In [281]:
r1.variables['time'][:]

array([ 0.5       ,  0.54166669,  0.58333331,  0.625     ,  0.66666669,
        0.70833331,  0.75      ,  0.79166669,  0.83333331,  0.875     ,
        0.91666669,  0.95833331,  1.        ], dtype=float32)